In [1]:
import os
import re
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
covid_repo_path = "..\\..\\COVID-19"
db_source = os.path.join(covid_repo_path, "csse_covid_19_data\\csse_covid_19_daily_reports")
print(f"list of files: {len(os.listdir(db_source))}")

list of files: 75


In [3]:
data_df = pd.DataFrame()
for file in tqdm(os.listdir(db_source)):
    try:
        crt_date, crt_ext = crt_file = file.split(".")
        if(crt_ext == "csv"):
            crt_date_df = pd.read_csv(os.path.join(db_source, file))
            crt_date_df['date_str'] = crt_date
            crt_date_df['Date'] = crt_date_df['date_str'].apply(lambda x: datetime.strptime(x, "%m-%d-%Y"))
            
            if(crt_date >=  '03-22-2020'):
                #break
                # FIPS, Admin2, Province_State, Country_Region, Last_Update,Lat, Long, Confirmed, Deaths, Recovered, Active, Combined_Key
                #print(f"Conversion start: {crt_date_df.columns}")
                crt_date_df['Province/State'] = crt_date_df['Province_State']
                crt_date_df['Country/Region'] = crt_date_df['Country_Region']
                crt_date_df['Latitude'] = crt_date_df['Lat']
                crt_date_df['Longitude'] = crt_date_df['Long_']
                crt_date_df = crt_date_df[['Country/Region', 'Province/State', 'Latitude', 'Longitude', 'Confirmed', 'Recovered', 'Deaths', 'Date', 'date_str']]
                
            data_df = data_df.append(crt_date_df)
    except ex as Exception:
        print(ex)
        pass

 37%|███▋      | 28/75 [00:00<00:00, 125.29it/s]C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
100%|██████████| 75/75 [00:01<00:00, 44.71it/s] 


In [4]:
print(f"Data: rows: {data_df.shape[0]}, cols: {data_df.shape[1]}")
print(f"Days: {data_df.date_str.nunique()} ({data_df.date_str.min()} : {data_df.date_str.max()})")
print(f"Country/Region: {data_df['Country/Region'].nunique()}")
print(f"Province/State: {data_df['Province/State'].nunique()}")
print(f"Confirmed all  (Province/State): {sum(data_df.groupby(['Province/State'])['Confirmed'].max())}")
print(f"Confirmed all (Country/Region): {sum(data_df.groupby(['Country/Region'])['Confirmed'].max())}")
print(f"Recovered all (Province/State): {sum(data_df.loc[~data_df.Recovered.isna()].groupby(['Province/State'])['Recovered'].max())}")
print(f"Recovered all (Country/Region): {sum(data_df.loc[~data_df.Recovered.isna()].groupby(['Country/Region'])['Recovered'].max())}")      
print(f"Deaths all (Province/State): {sum(data_df.loc[~data_df.Deaths.isna()].groupby(['Province/State'])['Deaths'].max())}")
print(f"Deaths all (Country/Region): {sum(data_df.loc[~data_df.Deaths.isna()].groupby(['Country/Region'])['Deaths'].max())}")

Data: rows: 48552, cols: 10
Days: 73 (01-22-2020 : 04-03-2020)
Country/Region: 231
Province/State: 295
Confirmed all  (Province/State): 262174.0
Confirmed all (Country/Region): 945121.0
Recovered all (Province/State): 90564.0
Recovered all (Country/Region): 263904.0
Deaths all (Province/State): 8619.0
Deaths all (Country/Region): 56463.0


In [5]:
data_df.head()

,Confirmed,Country/Region,Date,Deaths,Last Update,Latitude,Longitude,Province/State,Recovered,date_str
0,1.0,Mainland China,2020-01-22,NaN,1/22/2020 17:00,NaN,NaN,Anhui,NaN,01-22-2020
1,14.0,Mainland China,2020-01-22,NaN,1/22/2020 17:00,NaN,NaN,Beijing,NaN,01-22-2020
2,6.0,Mainland China,2020-01-22,NaN,1/22/2020 17:00,NaN,NaN,Chongqing,NaN,01-22-2020
3,1.0,Mainland China,2020-01-22,NaN,1/22/2020 17:00,NaN,NaN,Fujian,NaN,01-22-2020
4,NaN,Mainland China,2020-01-22,NaN,1/22/2020 17:00,NaN,NaN,Gansu,NaN,01-22-2020


In [6]:
data_df.tail()

,Confirmed,Country/Region,Date,Deaths,Last Update,Latitude,Longitude,Province/State,Recovered,date_str
2620,237.0,Vietnam,2020-04-03,0.0,NaN,14.058324,108.277199,NaN,85.0,04-03-2020
2621,194.0,West Bank and Gaza,2020-04-03,1.0,NaN,31.952200,35.233200,NaN,21.0,04-03-2020
2622,39.0,Zambia,2020-04-03,1.0,NaN,-13.133897,27.849332,NaN,2.0,04-03-2020
2623,9.0,Zimbabwe,2020-04-03,1.0,NaN,-19.015438,29.154857,NaN,0.0,04-03-2020
2624,0.0,US,2020-04-03,0.0,NaN,NaN,NaN,Wyoming,0.0,04-03-2020


In [ ]:
province_state = data_df['Province/State'].unique()

for ps in province_state:

    data_df.loc[(data_df['Province/State']==ps) & (data_df['Latitude'].isna()), 'Latitude'] =\
                data_df.loc[(~data_df['Latitude'].isna()) & \
                            (data_df['Province/State']==ps), 'Latitude'].median()
    
    data_df.loc[(data_df['Province/State']==ps) & (data_df['Longitude'].isna()), 'Longitude'] =\
            data_df.loc[(~data_df['Longitude'].isna()) & \
                        (data_df['Province/State']==ps), 'Longitude'].median() 

In [ ]:
country_region = data_df['Country/Region'].unique()

for cr in country_region:

    data_df.loc[(data_df['Country/Region']==cr) & (data_df['Latitude'].isna()), 'Latitude'] =\
                data_df.loc[(~data_df['Latitude'].isna()) & \
                            (data_df['Country/Region']==cr), 'Latitude'].median()
    
    data_df.loc[(data_df['Country/Region']==cr) & (data_df['Longitude'].isna()), 'Longitude'] =\
            data_df.loc[(~data_df['Longitude'].isna()) & \
                        (data_df['Country/Region']==cr), 'Longitude'].median() 

In [ ]:
data_df.loc[data_df.Latitude.isna()]

In [ ]:
data_df.loc[data_df.Longitude.isna()]

## Unify countries duplicates

In [ ]:
countries = list(data_df['Country/Region'].unique())
countries.sort()
print(countries)

In [ ]:
data_df.loc[data_df['Country/Region']==' Azerbaijan', 'Country/Region'] = 'Azerbaijan'
data_df.loc[data_df['Country/Region']=='Czechia', 'Country/Region'] = 'Czech Republic'
data_df.loc[data_df['Country/Region']=="Cote d'Ivoire", 'Country/Region'] = 'Ivory Coast'
data_df.loc[data_df['Country/Region']=="Dominica", 'Country/Region'] = 'Dominican Republic'
data_df.loc[data_df['Country/Region']=='Iran (Islamic Republic of)', 'Country/Region'] = 'Iran'
data_df.loc[data_df['Country/Region']=='Hong Kong SAR', 'Country/Region'] = 'Hong Kong'
data_df.loc[data_df['Country/Region']=='Holy See', 'Country/Region'] = 'Vatican City'
data_df.loc[data_df['Country/Region']=='Macao SAR', 'Country/Region'] = 'Macau'
data_df.loc[data_df['Country/Region']=='Mainland China', 'Country/Region'] = 'China'
data_df.loc[data_df['Country/Region']=='Niger', 'Country/Region'] = 'Nigeria'
data_df.loc[data_df['Country/Region']=='occupied Palestinian territory', 'Country/Region'] = 'West Bank and Gaza'
data_df.loc[data_df['Country/Region']=='Palestine', 'Country/Region'] = 'West Bank and Gaza'
data_df.loc[data_df['Country/Region']=='Republic of Ireland', 'Country/Region'] = 'Ireland'
data_df.loc[data_df['Country/Region']=='Korea, South', 'Country/Region'] = 'South Korea'
data_df.loc[data_df['Country/Region']=='Republic of Ireland', 'Country/Region'] = 'Ireland'
data_df.loc[data_df['Country/Region']=='Republic of Korea', 'Country/Region'] = 'South Korea'
data_df.loc[data_df['Country/Region']=='Republic of Moldova', 'Country/Region'] = 'Moldova'
data_df.loc[data_df['Country/Region']=='Republic of the Congo', 'Country/Region'] = 'Congo (Brazzaville)'
data_df.loc[data_df['Country/Region']=='Russian Federation', 'Country/Region'] = 'Russia'
data_df.loc[data_df['Country/Region']=='Taiwan*', 'Country/Region'] = 'Taiwan'
data_df.loc[data_df['Country/Region']=='The Gambia', 'Country/Region'] = 'Gambia'
data_df.loc[data_df['Country/Region']=='Gambia, The', 'Country/Region'] = 'Gambia'
data_df.loc[data_df['Country/Region']=='UK', 'Country/Region'] = 'United Kingdom'
data_df.loc[data_df['Country/Region']=='Viet Nam', 'Country/Region'] = 'Vietnam'


## Fix Lat/Long where missing

In [ ]:
data_df.loc[data_df['Country/Region']=='Ivory Coast', 'Longitude'] = 5.54
data_df.loc[data_df['Country/Region']=='Ivory Coast', 'Latitude'] = 7.54
data_df.loc[data_df['Country/Region']=='North Ireland', 'Longitude'] = 6.4923
data_df.loc[data_df['Country/Region']=='North Ireland', 'Latitude'] = 54.7877

In [ ]:
data_df = data_df[['Country/Region', 'Province/State', 'Latitude', 'Longitude', 'Confirmed', 'Recovered', 'Deaths', 'Date']]

In [ ]:
data_df.head()

## Check European Countries on 2020-03-12

Italy, Germany, France, Spain & other European Countries did not reported the data last days. Same data appears as for the previous day. We will fix using https://github.com/CSSEGISandData/COVID-19/issues/599 information, as following:

* Italy: 15113 confirmed cases, 1016 death and 1258 recovered (http://www.salute.gov.it/imgs/C_17_pagineAree_5351_8_file.pdf)  

* Spain: 2950 confirmed cases, 84 death
(https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/documentos/Actualizacion_42_COVID-19.pdf)  

* France: 2876 confirmed cases, 61 death (https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/infection-au-nouveau-coronavirus-sars-cov-2-covid-19-france-et-monde)  

* Switzerland: 815 confirmed cases, 4 death (https://www.bag.admin.ch/dam/bag/fr/dokumente/mt/k-und-i/aktuelle-ausbrueche-pandemien/2019-nCoV/covid-19-lagebericht.pdf.download.pdf/COVID-19_Situation_epidemiologique_en_Suisse.pdf)

* Germany : 2.369 confirmed, 5 deaths (https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html)  

* All EU countries: https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide  


In [ ]:
data_df[(data_df['Country/Region'].isin(['Italy', 'Spain', 'France', 'Switzerland', 'Germany', 'Netherlands'])) & (data_df['Date'].isin(['2020-03-11','2020-03-12', '2020-03-13']))]

In [ ]:
def fix_data_for_country_date(country, date, confirmed, recovered, deaths):
    data_df.loc[(data_df['Country/Region']==country) & (data_df['Date']==date), 'Confirmed'] = confirmed
    data_df.loc[(data_df['Country/Region']==country) & (data_df['Date']==date), 'Recovered'] = recovered
    data_df.loc[(data_df['Country/Region']==country) & (data_df['Date']==date), 'Deaths'] = deaths

In [ ]:
def fix_data_for_france_date(country, date, confirmed, recovered, deaths):
    data_df.loc[(data_df['Country/Region']==country) & (data_df['Province/State']==country) & (data_df['Date']==date), 'Confirmed'] = confirmed
    data_df.loc[(data_df['Country/Region']==country) & (data_df['Province/State']==country)  & (data_df['Date']==date), 'Recovered'] = recovered
    data_df.loc[(data_df['Country/Region']==country) & (data_df['Province/State']==country)  & (data_df['Date']==date), 'Deaths'] = deaths

In [ ]:
fix_data_for_country_date('Italy', '2020-03-12', 15113, 1258, 1016)
fix_data_for_country_date('Spain', '2020-03-12', 2950, 183, 84)
fix_data_for_france_date('France', '2020-03-12', 2896, 12, 61)
fix_data_for_country_date('Switzerland', '2020-03-12', 815, 4, 4)
fix_data_for_country_date('Germany', '2020-03-12', 2369, 25, 5)
fix_data_for_country_date('Netherlands', '2020-03-12', 614, 25, 5)

In [ ]:
data_df[(data_df['Country/Region'].isin(['Italy', 'Spain', 'France', 'Switzerland', 'Germany', 'Netherlands'])) & (data_df['Date'].isin(['2020-03-11','2020-03-12', '2020-03-13']))]

In [ ]:
data_df[(data_df['Country/Region'].isin(['France'])) & (data_df['Date'].isin(['2020-03-11','2020-03-12', '2020-03-13']))]

In [ ]:
data_df.to_csv("covid-19-all.csv", index=False)